In [1]:
from pylpex.utils import tokenize, parse, evaluate, format_ast
from pylpex import Interpreter
interpreter = Interpreter()

In [2]:

code = """
function f(a, b) {
    return a + b
}
array = [f]
array[0](1, 2)
"""

result = interpreter.evaluate(code)
print(result)

3


In [3]:
with open("tests/evaluator/function.txt", "r") as f:
    code = f.read()

print(code)

result = interpreter.evaluate(code)
print(result)




function factorial(n) {
    if n == 0 {
        return 1
    } else {
        return n * factorial(n - 1)
    }
}

factorial(5)
120


In [4]:
from tests.evaluator.unit_tests import get_test_cases, run_tests

tests = get_test_cases()
run_tests(tests)


------------------------------------------------
[1/83] // comment
	Result:    None
	Expected:  None
	Correct:  ✅
------------------------------------------------
[2/83] /* comment */
	Result:    None
	Expected:  None
	Correct:  ✅
------------------------------------------------
[3/83] /* multi
line
comment */
	Result:    None
	Expected:  None
	Correct:  ✅
------------------------------------------------
[4/83] none
	Result:    None
	Expected:  None
	Correct:  ✅
------------------------------------------------
[5/83] 45
	Result:    45
	Expected:  45
	Correct:  ✅
------------------------------------------------
[6/83] 4.5
	Result:    4.5
	Expected:  4.5
	Correct:  ✅
------------------------------------------------
[7/83] 'hello world'
	Result:    hello world
	Expected:  hello world
	Correct:  ✅
------------------------------------------------
[8/83] true
	Result:    True
	Expected:  True
	Correct:  ✅
------------------------------------------------
[9/83] false
	Result:    False
	Expect